In [1]:
!mkdir data
!wget http://cf.10xgenomics.com/samples/cell-exp/1.1.0/pbmc3k/pbmc3k_filtered_gene_bc_matrices.tar.gz -O data/pbmc3k_filtered_gene_bc_matrices.tar.gz
!cd data; tar -xzf pbmc3k_filtered_gene_bc_matrices.tar.gz

--2023-11-13 14:07:22--  http://cf.10xgenomics.com/samples/cell-exp/1.1.0/pbmc3k/pbmc3k_filtered_gene_bc_matrices.tar.gz
Resolving cf.10xgenomics.com (cf.10xgenomics.com)... 104.18.1.173, 104.18.0.173
Connecting to cf.10xgenomics.com (cf.10xgenomics.com)|104.18.1.173|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cf.10xgenomics.com/samples/cell-exp/1.1.0/pbmc3k/pbmc3k_filtered_gene_bc_matrices.tar.gz [following]
--2023-11-13 14:07:22--  https://cf.10xgenomics.com/samples/cell-exp/1.1.0/pbmc3k/pbmc3k_filtered_gene_bc_matrices.tar.gz
Connecting to cf.10xgenomics.com (cf.10xgenomics.com)|104.18.1.173|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7621991 (7.3M) [application/x-tar]
Saving to: 'data/pbmc3k_filtered_gene_bc_matrices.tar.gz'

data/pbmc3k_filtere 100%[===================>]   7.27M  24.3MB/s    in 0.3s    

2023-11-13 14:07:22 (24.3 MB/s) - 'data/pbmc3k_filtered_gene_bc_matrices.tar.gz' saved [7621991

In [2]:
import numpy as np
import pandas as pd
import scanpy as sc

sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

scanpy==1.9.3 anndata==0.9.1 umap==0.5.1 numpy==1.21.5 scipy==1.7.3 pandas==1.5.3 scikit-learn==1.0.2 statsmodels==0.13.5 pynndescent==0.5.10


In [3]:
adata = sc.read_10x_mtx(
    'data/filtered_gene_bc_matrices/hg19/',  # the directory with the `.mtx` file
    var_names='gene_symbols',                # use gene symbols for the variable names (variables-axis index)
    cache=True)           

adata.var_names_make_unique()
adata

... writing an h5ad cache file to speedup reading next time


AnnData object with n_obs × n_vars = 2700 × 32738
    var: 'gene_ids'

In [4]:
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)

filtered out 19024 genes that are detected in less than 3 cells


In [5]:
adata.var['mt'] = adata.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

In [6]:
adata.obs.head(10)

,n_genes,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt
AAACATACAACCAC-1,781,779,2419.0,73.0,3.017776
AAACATTGAGCTAC-1,1352,1352,4903.0,186.0,3.793596
AAACATTGATCAGC-1,1131,1129,3147.0,28.0,0.889736
AAACCGTGCTTCCG-1,960,960,2639.0,46.0,1.743085
AAACCGTGTATGCG-1,522,521,980.0,12.0,1.224490
AAACGCACTGGTAC-1,782,781,2163.0,36.0,1.664355
AAACGCTGACCAGT-1,783,782,2175.0,83.0,3.816092
AAACGCTGGTTCTT-1,790,790,2260.0,70.0,3.097345
AAACGCTGTAGCCA-1,533,532,1275.0,15.0,1.176471
AAACGCTGTTTCTG-1,550,550,1103.0,32.0,2.901179


In [10]:
adata.var.head(10)

,gene_ids,n_cells,mt,n_cells_by_counts,mean_counts,pct_dropout_by_counts,total_counts
AL627309.1,ENSG00000237683,9,False,9,0.003333,99.666667,9.0
AP006222.2,ENSG00000228463,3,False,3,0.001111,99.888889,3.0
RP11-206L10.2,ENSG00000228327,5,False,5,0.001852,99.814815,5.0
RP11-206L10.9,ENSG00000237491,3,False,3,0.001111,99.888889,3.0
LINC00115,ENSG00000225880,18,False,18,0.006667,99.333333,18.0
NOC2L,ENSG00000188976,258,False,258,0.106667,90.444444,288.0
KLHL17,ENSG00000187961,9,False,9,0.003333,99.666667,9.0
PLEKHN1,ENSG00000187583,7,False,7,0.002593,99.740741,7.0
RP11-54O7.17,ENSG00000272512,3,False,3,0.001111,99.888889,3.0
HES4,ENSG00000188290,145,False,145,0.078889,94.629630,213.0


In [11]:
# row sum of adata.X
rows_sum = adata.X.sum(axis=1)
rows_sum[:10]  # equal to total_counts

matrix([[2419.],
        [4903.],
        [3147.],
        [2639.],
        [ 980.],
        [2163.],
        [2175.],
        [2260.],
        [1275.],
        [1103.]], dtype=float32)

In [16]:
arr = adata.X.toarray()
# non zero values of the first row in arr
arr[0][arr[0].nonzero()]

array([ 1.,  1.,  2.,  1.,  1.,  1.,  1., 41.,  1.,  1.,  1.,  1.,  2.,
        6.,  1.,  1.,  1.,  1.,  1.,  1.,  2.,  1., 15.,  1.,  1.,  1.,
        1.,  1.,  1., 11.,  1.,  1.,  5.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  3.,  2.,  1., 15.,  2.,  1.,  1.,
        1.,  1.,  1.,  4.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  2.,  1.,
        1.,  1.,  1.,  1.,  1.,  1., 13.,  1.,  1.,  1.,  1.,  2.,  1.,
        5.,  2.,  2., 22.,  1.,  1.,  2.,  1.,  1.,  1.,  2.,  6.,  1.,
        1.,  1.,  2.,  1.,  1.,  1.,  1., 11.,  1.,  1.,  1.,  2.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  3.,  1.,  2.,  1.,  2.,  1.,
        4.,  1.,  1.,  1.,  1., 11.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  2., 17.,  1.,  8.,  1., 17., 11.,  1.,  1.,  1.,  1.,
        1., 14.,  1.,  2.,  1.,  1.,  1.,  1.,  5.,  1.,  1.,  1.,  1.,
        1.,  1.,  2.,  1.,  2.,  1.,  3.,  1.,  1.,  7.,  1.,  2.,  1.,
        1., 23.,  1.,  1.,  1.,  1.,  1.,  2.,  1.,  2.,  1.,  1